In [1]:
import pandas as pd
import numpy as np

In [20]:
from influxdb import *
INFLUX_DB_IP = '10.12.97.178'
INFLUX_DB_PORT = 8086
INFLUX_DB = 'demoDb'
# READ_MEASUREMENT = 'EM_main'
# LIVE_MEASUREMENT = 'EM_live'
TAG_MEASUREMENT = 'EM_main_new'

con_obj = InfluxDBClient(host=INFLUX_DB_IP, port=INFLUX_DB_PORT, database=INFLUX_DB)
query = 'select * from ' + str(TAG_MEASUREMENT) + ' where time > now() - 1d '
df = pd.DataFrame(con_obj.query(query, chunked=True, chunk_size=10000).get_points())
df['time'] = pd.to_datetime(df['time']) 

In [21]:
df.head()

,time,DeviceID,EM_Active Power (kW),EM_Current Avg (A),EM_Current Ph1 (A),EM_Current Ph2 (A),EM_Current Ph3 (A),EM_Energy (kWh),EM_Frequency (Hz),EM_Power Factor,...,EM_Voltage Ph2-N (V),EM_Voltage Ph2-Ph3 (V),EM_Voltage Ph3-N (V),Mean_THD,imbalance_current,imbalance_volt,mean_current,mean_volt,mean_voltage,temp_rise
0,2020-05-26 10:27:22.130095872+00:00,EM23,0.0,0.0,0.0,0.0,0.0,69873.9,50.0,0.0,...,236.3,411.6,238.6,0.000000,0.000000,0.040489,0.000000,411.633333,237.633333,0.003279
1,2020-05-26 10:27:22.504299008+00:00,EM2,0.0,0.0,0.0,0.0,0.0,2532.0,50.0,1.0,...,243.3,420.6,244.2,0.000000,0.000000,0.395570,0.000000,421.333333,243.233333,0.312951
2,2020-05-26 10:27:22.740067072+00:00,EM10,81462.4,193.5,206.8,193.0,180.6,122283.7,50.0,0.9,...,236.2,408.7,234.9,0.000000,6.891799,0.318315,193.466667,408.400000,235.800000,0.202649
3,2020-05-26 10:27:22.789952+00:00,EM6,630.0,1.4,1.4,1.4,1.4,268963.3,50.0,0.6,...,227.6,392.8,227.0,0.000000,0.000000,0.178208,1.400000,392.800000,226.766667,0.063516
4,2020-05-26 10:27:23.303179008+00:00,EM4,438.7,670.7,646.5,670.6,695.1,117039.0,50.0,0.9,...,227.5,393.0,226.7,7.766667,3.632840,0.076394,670.733333,392.700000,226.700000,0.011672


In [17]:
class agg():

    def __init__(self, Upper_threshold, lower_threshold):
        self.Upper_threshold = Upper_threshold
        self.lower_threshold = lower_threshold

    def set_val(self, row):
        if row['EM_Voltage Ph1-N (V)'] > self.Upper_threshold:
            return "high"
        elif row['EM_Voltage Ph1-N (V)'] < self.lower_threshold:
            return "low"
        else:
            return "medium"

    def daily(self, df1):
        A = []
        df0 = df1[(df1['EM_Voltage Ph1-N (V)'] == 0)]
        df1 = df1[(df1['EM_Voltage Ph1-N (V)'] > 0)]

        time = df1['time'].max()

        MEAN = df1['EM_Voltage Ph1-N (V)'].mean()
        MAX = df1['EM_Voltage Ph1-N (V)'].max()
        MIN = df1['EM_Voltage Ph1-N (V)'].min()
        quantile25 = df1['EM_Voltage Ph1-N (V)'].quantile(0.25)
        quantile75 = df1['EM_Voltage Ph1-N (V)'].quantile(0.75)
        IQR = quantile75 - quantile25
        SD = df1['EM_Voltage Ph1-N (V)'].std()

        df1 = df1.assign(flag2=df1.apply(self.set_val, axis=1))
        y = pd.DataFrame(df1.groupby([(df1.flag2 != df1.flag2.shift()).cumsum()])['time'].apply(lambda y: (y.iloc[-1] - y.iloc[0]).total_seconds() / 60))
        y['flag2'] = df1.loc[df1.flag2.shift(-1) != df1.flag2]['flag2'].values
        y.reset_index(drop=True, inplace=True)

        duration_high = y[y['flag2'] == "high"].time.sum()
        duration_medium = y[y['flag2'] == "medium"].time.sum()
        duration_low = y[y['flag2'] == "low"].time.sum()

        duration_0 = df0.groupby(df0.index.to_series().diff().ne(1).cumsum())['time'].agg(
            lambda x: abs((x.iloc[-1] - x.iloc[0]).total_seconds() / 60)).sum()

        non_zeros = len(df1)
        zeros = len(df0)
        overlimit_count = len(df1[df1['flag2'] == "high"])
        underlimit_count = len(df1[df1['flag2'] == "low"])
        onlimit_count = len(df1[df1['flag2'] == "medium"])

        df1['voltage_difference'] = df1['EM_Voltage Ph1-N (V)'].diff(-1)
        Average_jump = abs(df1['voltage_difference']).mean()
        min_jump = abs(df1['voltage_difference']).min()
        max_jump = abs(df1['voltage_difference']).max()
        df1['jumps_instantaneous'] = np.where(abs(df1["voltage_difference"]) > (Average_jump * 1.1), 1, 0)
        jumps_instantaneous = len(df1[df1['jumps_instantaneous'] == 1])

        df1['delta_volt'] = df1['EM_Voltage Ph1-N (V)'] - MEAN
        df1['jumps_average'] = np.where(abs(df1["delta_volt"]) > (abs(df1['delta_volt']).mean() * 1.1), 1, 0)
        jumps_average = len(df1[df1['jumps_average'] == 1])

        upper_difference = MEAN - self.Upper_threshold
        lower_difference = MEAN - self.lower_threshold

        A = [time, MEAN, MAX, MIN, quantile25, quantile75, IQR, SD, upper_difference, lower_difference, duration_high,
             duration_medium, duration_low, duration_0, non_zeros, zeros, overlimit_count, underlimit_count,
             onlimit_count, Average_jump, min_jump, max_jump, jumps_instantaneous, jumps_average]
        FINAL = pd.DataFrame(
            columns=['time', 'MEAN', 'MAX', 'MIN', 'quantile25', 'quantile75', 'IQR', 'SD', 'upper_difference',
                     'lower_difference', 'duration_high', 'duration_medium', 'duration_low', 'duration_0', 'non_zeros',
                     'zeros', 'overlimit_count', 'underlimit_count', 'onlimit_count', 'Average_jump', 'min_jump',
                     'max_jump', 'jumps_instantaneous', 'jumps_average'], data=np.array(A).reshape(-1, len(A)))
        return FINAL

    def sensor(self, df):
        final_sen = df.groupby('DeviceID').apply(self.daily)
        final_sen.reset_index(inplace=True)
        final_sen.drop(final_sen.columns[1], axis=1, inplace=True)
        final_sen.set_index('time', inplace = True) 
        return final_sen

In [18]:
agg1 = agg(245,190)
agg1.sensor(df[['time','EM_Voltage Ph1-N (V)','DeviceID']])

,DeviceID,MEAN,MAX,MIN,quantile25,quantile75,IQR,SD,upper_difference,lower_difference,...,non_zeros,zeros,overlimit_count,underlimit_count,onlimit_count,Average_jump,min_jump,max_jump,jumps_instantaneous,jumps_average
time,,,,,,,,,,,,,,,,,,,,,
2020-05-26 07:27:05.360854016+00:00,EM1,247.249,257.1,236.4,243.8,251.8,8,5.11524,2.24857,57.2486,...,31348,0,20240,0,11108,0.0465212,0,3.8,11902,13909
2020-05-26 07:27:07.066180864+00:00,EM10,239.914,240.9,236.5,240.4,240.6,0.2,1.34283,-5.08649,49.9135,...,30852,0,0,0,30852,0.023996,0,2,7337,6183
2020-05-26 07:27:05.830708992+00:00,EM2,247.255,257.3,235.9,243.9,252,8.1,5.34242,2.2553,57.2553,...,33241,0,21836,0,11405,0.0447473,0,3.6,11822,14407
2020-05-26 07:27:05.517099008+00:00,EM22,229.255,230.8,223.1,227.8,230.6,2.8,2.07239,-15.7448,39.2552,...,30863,0,0,0,30863,0.0205787,0,2.1,6198,7044
2020-05-26 07:27:05.392105984+00:00,EM23,240.267,241,236.9,240.5,240.7,0.2,0.857482,-4.73292,50.2671,...,30804,0,0,0,30804,0.0428627,0,2,8620,6268
2020-05-26 07:27:04.078481920+00:00,EM3,246.979,257,234.2,243.6,251.7,8.1,5.33649,1.97892,56.9789,...,26522,0,16422,0,10100,0.0505411,0,4.9,10105,11495
2020-05-26 07:27:06.143192064+00:00,EM4,228.638,229.6,225.3,228.9,229.4,0.5,1.18756,-16.3621,38.6379,...,26495,0,0,0,26495,0.0763229,0,1.5,14320,6112
2020-05-26 07:27:05.438977024+00:00,EM5,229.643,240.7,225.4,229.8,230.7,0.9,1.69915,-15.3567,39.6433,...,31905,0,0,0,31905,0.0603216,0,10.3,16091,7266
2020-05-26 07:27:06.236942080+00:00,EM6,228.864,229.8,225,229,229.7,0.7,1.27136,-16.1359,38.8641,...,31011,0,0,0,31011,0.0243663,0,1.9,6795,7411
